In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Vine-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

Deliverable 2:  Vine Analysis

In [12]:
# Confirm that the columns for total_vote, helpful_votes, and star_rating all contain integers:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [14]:
# 1) Filter furniture data frame on total_vote greater than or equal to 20
vote_count_df = df.filter("total_votes>=20")
vote_count_df.show(5)
vote_count_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

18739

In [26]:
# 2) Filter dataframe to retrieve rows where helpful votes/ total votes is greater than 50%:
helpful_df = vote_count_df.filter("helpful_votes/total_votes>=.50")
helpful_df.show(5)
helpful_df.count()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

18155

In [36]:
# 3) Filter dataframe to retrieve rows where review was written as part of the Vine program
vine_y_df = helpful_df.filter(helpful_df["vine"] == "Y")
vine_y_df.show(5)
vine_y_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35119071|R2BQOD1R0228FN|B00H2RSA88|     405483618|Sleep Innovations...|       Furniture|          3|           17|         26|   Y|                N|An okay product. ...|Three-stars is co...| 2015-08-27|
|         US|   44737123| RC31RUPFOHBHQ|B0125QZ50G|     350975212|Zinus Viscolatex ...|       Furniture|          4|    

136

In [28]:
# 3) Filter dataframe to retrieve rows where review was NOT written as part of the Vine program:
vine_n_df = helpful_df.filter(helpful_df["vine"] == "N")
vine_n_df.show(5)
vine_n_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

18019

In [34]:
# 5.A) Total number of ALL reviews:
total_review_count = df.count()
print("Total Number of Reviews: %f" % total_review_count)

Total Number of Reviews: 792113.000000


In [42]:
# 5.B) Total number of reviews for each dataframe from Step 3 and Step 4:
y_vine_count = vine_y_df.count()
print("Total Number of Reviews that are  PAID and 'helpful': %f" % y_vine_count)

n_vine_count = vine_n_df.count()
print("Total Number of Reviews that are UNPAID and 'helpful': %f" % n_vine_count)

Total Number of Reviews that are  PAID and 'helpful': 136.000000
Total Number of Reviews that are UNPAID and 'helpful': 18019.000000


In [43]:
# 5.C) Number of 5-star Reviews for above PAID and UNPAID dataframes
fivestar_y_df = vine_y_df.filter(vine_y_df["star_rating"]=="5")
print("Total Number of 5-star Reviews that are  PAID and 'helpful': %f" % fivestar_y_df.count())

fivestar_n_df = vine_n_df.filter(vine_n_df["star_rating"]=="5")
print("Total Number of 5-star Reviews that are  UNPAID and 'helpful': %f" % fivestar_n_df.count())

Total Number of 5-star Reviews that are  PAID and 'helpful': 74.000000
Total Number of 5-star Reviews that are  UNPAID and 'helpful': 8482.000000


In [49]:
# 5.D) Percentage of 5-star Reviews for above PAID and UNPAID dataframes
fivestar_y_percent = (fivestar_y_df.count()/y_vine_count)*100
print("Percent of 5-Star Furniture Reviews that are PAID and 'helpful': %f" % fivestar_y_percent)

fivestar_n_percent = (fivestar_n_df.count()/n_vine_count)*100
print("Percent of 5-Star Furniture Reviews that are UNPAID and 'helpful':  %f" % fivestar_n_percent)

Percent of 5-Star Furniture Reviews that are PAID and 'helpful': 54.411765
Percent of 5-Star Furniture Reviews that are UNPAID and 'helpful':  47.072535
